<a href="https://colab.research.google.com/github/Jessica1842553/MCDdm2026/blob/main/Tarea2_DM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tarea 2 - **Operaciones con RDDs**
### Jessica Lizeth Hernández Bracho - 1842553

**Se trabajó con el dataset MovieLens 10M, el cual contiene millones de calificaciones de usuarios sobre películas, permitiendo aplicar técnicas de procesamiento distribuido mediante RDDs en PySpark.**

Se calcularon estadísticas descriptivas básicas, como el rating promedio por película, empleando transformaciones como map y reduceByKey. Posteriormente, se realizó una unión (join) entre los ratings y los metadatos de las películas para analizar el rating promedio por género, utilizando flatMap para manejar películas con múltiples géneros.

In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext("local", "TareaRDD")

In [3]:
from pyspark.sql import SparkSession
# Descarga de MovieLens 10M dataset
!wget -q --show-progress http://files.grouplens.org/datasets/movielens/ml-10m.zip -P /content/
!unzip -q /content/ml-10m.zip -d /content/

spark = SparkSession.builder.getOrCreate()

ratings = spark.read \
    .option("delimiter", "::") \
    .option("header", "false") \
    .csv(
        "/content/ml-10M100K/ratings.dat",
        schema="userId INT, movieId INT, rating DOUBLE, timestamp LONG"
    )

movies = spark.read \
    .option("delimiter", "::") \
    .option("header", "false") \
    .csv(
        "/content/ml-10M100K/movies.dat",
        schema="movieId INT, title STRING, genres STRING"
    )

tags = spark.read \
    .option("delimiter", "::") \
    .option("header", "false") \
    .csv(
        "/content/ml-10M100K/tags.dat",
        schema="userId INT, movieId INT, tag STRING, timestamp LONG"
    )

ml-10m.zip          100%[===================>]  62.53M  4.72MB/s    in 10s     


In [4]:
ruta = "/content/ml-10M100K/ratings.dat"

In [5]:
lines = sc.textFile(ruta)

In [6]:
rdd_ratings = lines.map(lambda l: l.split("::")) \
                   .map(lambda p: (int(p[0]), int(p[1]), float(p[2]), int(p[3])))

In [7]:
rdd_ratings.cache()

PythonRDD[2] at RDD at PythonRDD.scala:56

In [8]:
rdd_ratings.take(5)

[(1, 122, 5.0, 838985046),
 (1, 185, 5.0, 838983525),
 (1, 231, 5.0, 838983392),
 (1, 292, 5.0, 838983421),
 (1, 316, 5.0, 838983392)]

### Estadísticas Básicas

In [9]:
# TOTAL DE CALIFICACIONES
total_ratings = rdd_ratings.count()
print("Total de ratings:", total_ratings)

Total de ratings: 10000054


In [10]:
# Suma, mínimo, máximo y promedio de ratings
ratings_only = rdd_ratings.map(lambda x: x[2])

min_rating = ratings_only.min()
max_rating = ratings_only.max()
mean_rating = ratings_only.mean()

print("Mínimo rating:", min_rating)
print("Máximo rating:", max_rating)
print("Promedio rating:", mean_rating)

Mínimo rating: 0.5
Máximo rating: 5.0
Promedio rating: 3.512421932921543


In [11]:
# Número de ratings por película
ratings_ppelicula = rdd_ratings.map(lambda x: (x[1], 1)).reduceByKey(lambda a, b: a + b)

# Mostrar las 10 películas con más ratings
top10 = ratings_ppelicula.takeOrdered(10, key=lambda x: -x[1])
print(top10)

[(296, 34864), (356, 34457), (593, 33668), (480, 32631), (318, 31126), (110, 29154), (457, 28951), (589, 28948), (260, 28566), (150, 27035)]


### Rating promedio por película

In [12]:
ratings_path = "/content/ml-10M100K/ratings.dat"


In [13]:
ratings_raw = sc.textFile(ratings_path)

In [14]:
ratings = ratings_raw.map(lambda l: l.split("::")).map(lambda x: (int(x[1]), float(x[2])))

In [15]:
ratings.cache()

PythonRDD[15] at RDD at PythonRDD.scala:56

In [16]:
# Calcular promedio por película
ratings_sum_count = ratings.mapValues(lambda r: (r, 1)).reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))

ratings_promedio_pelicula = ratings_sum_count.mapValues(lambda x: x[0] / x[1])

ratings_promedio_pelicula.take(10)

[(480, 3.661564156783427),
 (520, 3.0130860828578516),
 (616, 3.3823024054982818),
 (376, 3.278003482298317),
 (648, 3.386744130898743),
 (736, 3.2202010531354714),
 (1544, 2.940903540903541),
 (1288, 4.071249715456408),
 (1408, 3.717517319383571),
 (1552, 3.1715748230535894)]

### Promedio de ratings por género

In [17]:
movies_path = "/content/ml-10M100K/movies.dat"

movies_raw = sc.textFile(movies_path)

movies = movies_raw.map(lambda l: l.split("::")).map(lambda x: (int(x[0]), x[2].split("|")))

In [18]:
movies.cache()

PythonRDD[23] at RDD at PythonRDD.scala:56

In [19]:
# Unir ratings con géneros (JOIN)
ratings_mmovies = ratings.join(movies)

In [21]:
ratings_mmovies.cache()

PythonRDD[31] at RDD at PythonRDD.scala:56

In [22]:
# Una película puede tener varios géneros
ratings_por_genero = ratings_mmovies.flatMap(lambda x: [(genero, x[1][0]) for genero in x[1][1]])

In [23]:
# Calcular promedio por género
genero_sum_count = ratings_por_genero.mapValues(lambda r: (r, 1)).reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))

promedio_por_genero = genero_sum_count.mapValues(lambda x: x[0] / x[1])

promedio_por_genero.collect()

[('Fantasy', 3.502019481138221),
 ('Action', 3.4213307400955033),
 ('Thriller', 3.5071860967677653),
 ('War', 3.7801731498090883),
 ('Animation', 3.5999880565273004),
 ('Romance', 3.553776441558182),
 ('IMAX', 3.7645374449339206),
 ('Crime', 3.6656546597629625),
 ('Children', 3.4184739602194236),
 ('Mystery', 3.6776306613582186),
 ('Musical', 3.56247843815335),
 ('Western', 3.555656921300586),
 ('Horror', 3.269243385726838),
 ('Drama', 3.6732628208935227),
 ('Sci-Fi', 3.396193464024223),
 ('Adventure', 3.4936211560747057),
 ('Comedy', 3.436946311044954),
 ('Film-Noir', 4.012151194601495),
 ('Documentary', 3.7834593152512226),
 ('(no genres listed)', 3.642857142857143)]

In [24]:
promedios_ordenados = promedio_por_genero.sortBy(lambda x: -x[1])

promedios_ordenados.collect()

[('Film-Noir', 4.012151194601495),
 ('Documentary', 3.7834593152512226),
 ('War', 3.7801731498090883),
 ('IMAX', 3.7645374449339206),
 ('Mystery', 3.6776306613582186),
 ('Drama', 3.6732628208935227),
 ('Crime', 3.6656546597629625),
 ('(no genres listed)', 3.642857142857143),
 ('Animation', 3.5999880565273004),
 ('Musical', 3.56247843815335),
 ('Western', 3.555656921300586),
 ('Romance', 3.553776441558182),
 ('Thriller', 3.5071860967677653),
 ('Fantasy', 3.502019481138221),
 ('Adventure', 3.4936211560747057),
 ('Comedy', 3.436946311044954),
 ('Action', 3.4213307400955033),
 ('Children', 3.4184739602194236),
 ('Sci-Fi', 3.396193464024223),
 ('Horror', 3.269243385726838)]

### Operaciones realizadas con RDDs

- Cálculo del rating promedio por película usando `map`, `reduceByKey`
- Unión de ratings con metadatos de películas usando `join`
- Expansión de múltiples géneros por película con `flatMap`
- Cálculo del rating promedio por género